# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import os

# Funciones

In [249]:
def get_hours(x,y):
    val = y.replace(' ','')
    if val[-1]=='h':
        return float(val[:-1])
    elif val.isdigit():
        return float(val)
    
    val = x.replace(' ','')
    if val[-1]=='h':
        return float(val[:-1])
    elif val.isdigit():
        return float(val)
    else:
        return np.nan

# Ruta de archivos

In [13]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_etl = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))

# Costos de cursos

# Funciones

In [40]:
def set_name_area(x):
    if x in ['administracion','administracion y negocios','negocios y administracion','administración y legislación\n','empresariales','gestion empresarial','facultad de cienciasadministrativas','faculta de jurisprudencia ,  ciencias']:
        return 'administración y legislación'
    elif x in ['salud','facultad ciencias ce la salud']:
        return 'actividades de salud'
    elif x in ['tecnologicos','tecnologia','tecnologia e innovacion','tecnologia e informacion','tics','tecnología de la información y comunicación',' tecnología de la información y comunicación']:
        return 'tecnologías de la información y comunicación'
    elif x in ['educativos','educacion','facultad de ciencias de la educacion']:
        return 'educación y capacitación'
    elif x in ['mecanica automotriz']:
        return 'mecánica automotriz'
    elif x in ['social']:
        return 'trabajo social'
    elif x in ['financiero','finanzas, comercio y ventas']:
        return 'servicios financieros'
    elif x in ['facultad  de ciencias agropecuarias']:
        return 'industria agropecuaria'
    elif x in ['agronomo']:
        return 'agronomía'
    elif x in ['comunicación']:
        return 'comunicación y artes gráficas'
    elif x in ['arte']:
        return 'artes y artesanía'
    else:
        return 'area base'
    

In [106]:
def set_name_modalidad(x):
    if 'online' in x or 'on line' in x:
        return 'online'
    if 'presencial' in x:
        return 'presencial'
    if 'virtual' in x or 'en vivo' in x:
        return 'virtual'
    else:
        return x
    

In [78]:
def set_precio(x):
    x = x.replace('$','')
    x = x.replace(',','')
    x = x.replace(' ','')
    x = x.replace('-','')
    
    if 'llamar' in x:
        return np.nan
    if ''==x:
        return np.nan
    return float(x)
    

In [283]:
def fill_precio_na(x,y):   
    if pd.isna(x['precio_set']):
        if x['modalidad_set'] in y['mean_by_mod'].keys():
            x['mean_by_mod'] = y['mean_by_mod'][x['modalidad_set']]
            
        if x['area_familia'] in y['mean_by_area'].keys():
            x['mean_by_area'] =  y['mean_by_area'][x['area_familia']]
            
        if x['Ciudad'] in y['mean_by_ciudad'].keys():
            x['mean_by_ciudad'] =  y['mean_by_ciudad'][x['Ciudad']]
            
            
        if x['modalidad_set']+'|'+x['area_familia'] in y['mean_by_ma'].keys():
            x['mean_by_ma'] =  y['mean_by_ma'][x['modalidad_set']+'|'+x['area_familia']]
            
        if x['area_familia']+'|'+x['Ciudad'] in y['mean_by_ac'].keys():
            x['mean_by_ac'] =  y['mean_by_ac'][x['area_familia']+'|'+x['Ciudad']]
            
        if x['modalidad_set']+'|'+x['Ciudad'] in y['mean_by_mc'].keys():
            x['mean_by_mc'] =  y['mean_by_mc'][x['modalidad_set']+'|'+x['Ciudad']]
            
            
        #if x['area_familia']+'|'+x['modalidad_set']+'|'+x['Ciudad'] in y['mean_by_mac'].keys():
        #    x['mean_by_mac'] =  y['mean_by_mac'][x['area_familia']+'|'+x['modalidad_set']+'|'+x['Ciudad']]
    return x

In [278]:
# Selecting Data
df_coec = pd.read_csv(os.path.join(path_additional_data,'costo_oferta_educacion_continua.csv'))
df_coec = df_coec[['Ciudad','Areas de Formación', 'Nombre  de los Programas','Modalidad', 'Tiempo de duración','Duración programa',' Precio ']]
df_coec = df_coec.rename(columns={' Precio ':'Precio'})
for col in df_coec.columns:
    df_coec[col] = df_coec[col].astype('str').str.lower()
    aux = df_coec[col][0]
    for i in range(df_coec.shape[0]):
        if pd.isna(df_coec[col][i]) or df_coec[col][i]=='nan' :
            df_coec[col][i] = aux
        else:
            aux = df_coec[col][i]
            


In [269]:
mean_by_mod = {}
mean_by_area = {}
mean_by_ciudad = {}
mean_by_ma = {}
mean_by_ac = {}
mean_by_mc = {}
mean_by_mac = {}
mean_list = {'mean_by_mod':mean_by_mod,'mean_by_area':mean_by_area,'mean_by_ciudad':mean_by_ciudad,'mean_by_ma':mean_by_ma,'mean_by_ac':mean_by_ac,'mean_by_mc':mean_by_mc}
for val in df_coec['modalidad_set'].value_counts().index.array:
    mean_by_mod[val]=df_coec[df_coec['modalidad_set']==val]['precio_set'].mean()
    for val2 in df_coec['area_familia'].value_counts().index.array:
        mean_by_ma[val+'|'+val2]=df_coec[(df_coec['area_familia']==val2)&(df_coec['modalidad_set']==val)]['precio_set'].mean()
        for val3 in df_coec['Ciudad'].value_counts().index.array:
            mean_by_mac[val+'|'+val2+'|'+val3]=df_coec[(df_coec['area_familia']==val2)&(df_coec['modalidad_set']==val)&(df_coec['Ciudad']==val3)]['precio_set'].mean()
        
        
for val in df_coec['area_familia'].value_counts().index.array:
    mean_by_area[val]=df_coec[df_coec['area_familia']==val]['precio_set'].mean()
    for val2 in df_coec['Ciudad'].value_counts().index.array:
        mean_by_ac[val+'|'+val2]=df_coec[(df_coec['Ciudad']==val2)&(df_coec['area_familia']==val)]['precio_set'].mean()
        
for val in df_coec['Ciudad'].value_counts().index.array:
    mean_by_ciudad[val]=df_coec[df_coec['Ciudad']==val]['precio_set'].mean()
    for val2 in df_coec['modalidad_set'].value_counts().index.array:
        mean_by_mc[val2+'|'+val]=df_coec[(df_coec['modalidad_set']==val2)&(df_coec['Ciudad']==val)]['precio_set'].mean()

In [279]:
df_coec['area_familia'] = df_coec.apply(lambda x: set_name_area(x['Areas de Formación']), axis=1 )

In [280]:
df_coec['modalidad_set'] = df_coec.apply(lambda x: set_name_modalidad(x['Modalidad']), axis=1 )

In [281]:
df_coec['precio_set'] = df_coec.apply(lambda x: set_precio(x['Precio']), axis=1 )

In [284]:
df_coec = df_coec.apply(lambda x: fill_precio_na(x,mean_list), axis=1)

In [285]:
df_coec['carga_horaria'] = df_coec.apply(lambda x: get_hours(x['Tiempo de duración'], x['Duración programa']), axis=1)

In [297]:
df_coec['carga_horaria'] = df_coec['carga_horaria'].fillna(df_coec['carga_horaria'].mean())

In [298]:
df_coec['precio_por_hora'] = df_coec.apply(lambda x: np.round(x['precio_set']/x['carga_horaria'],decimals = 2), axis=1)
df_coec['precio_por_hora'] = df_coec['precio_por_hora'].fillna(df_coec['precio_por_hora'].mean())

In [308]:
def get_prices_by(x,y):
    for mean_by in y.keys():
        x['precio_'+mean_by] =  np.round(x[mean_by]/x['carga_horaria'],decimals = 2)
    return x

In [309]:
df_coec = df_coec.apply(lambda x: get_prices_by(x,mean_list), axis=1)

In [325]:
df_coec_final = df_coec[['Ciudad','area_familia','Nombre  de los Programas','modalidad_set','carga_horaria','precio_por_hora','precio_mean_by_mod','precio_mean_by_area','precio_mean_by_ciudad',	'precio_mean_by_ma','precio_mean_by_ac','precio_mean_by_mc']]

In [328]:
df_coec_final = df_coec_final.rename(columns={'Ciudad':'ciudad'})
df_coec_final = df_coec_final.rename(columns={'Nombre  de los Programas':'cursos'})

In [329]:
df_coec_final.head(10)

,ciudad,area_familia,cursos,modalidad_set,carga_horaria,precio_por_hora,precio_mean_by_mod,precio_mean_by_area,precio_mean_by_ciudad,precio_mean_by_ma,precio_mean_by_ac,precio_mean_by_mc
0,quito,administración y legislación,gerencia de instituciones de salud,online,1736.0,6.875873,0.10,0.21,0.1,0.07,0.06,0.18
1,quito,educación y capacitación,liderazgo educativo,online,1680.0,6.875873,0.10,0.08,0.1,0.12,0.05,0.18
2,quito,actividades de salud,seguridad y salud ocupacional,online,1680.0,6.875873,0.10,0.31,0.1,0.04,0.54,0.18
3,quito,administración y legislación,administracion de empresas,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
4,quito,administración y legislación,desarrollo de negocios y emprendimiento,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
5,quito,administración y legislación,direccion estrategica,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
6,quito,administración y legislación,e-commerce,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
7,quito,administración y legislación,gestion de negocios con tecnologias digitrales,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
8,quito,administración y legislación,gestion de riesgos y manejo de crisis en el se...,online,168.0,6.875873,1.01,2.15,1.0,0.67,0.67,1.84
9,quito,tecnologías de la información y comunicación,big data,online,168.0,6.875873,1.01,1.91,1.0,1.59,1.25,1.84


In [347]:
df_coec_final.to_csv(os.path.join(path_etl,'costos_cursos_educacion_continua.csv'),index=False)

# Ruta de archivos - PROCESO de KPI de Asistencia

In [ ]:
path = os.path.join(os.getcwd(),'DATAWAREHOUSE')
path_dmt_oc = os.path.join(path,'datamart_oc')
path_dmt_oec = os.path.join(path,'datamart_oec')
path_dmt_ci = os.path.join(path,'datamart_ci')

## Analisis OC

In [ ]:
df_fact_oc = pd.read_csv(os.path.join(path_dmt_oc,'fact_oc.csv'))
df_dim_razon_social_oc = pd.read_csv(os.path.join(path_dmt_oc,'dim_razon_social_oc.csv'))
df_dim_curso_perfil = pd.read_csv(os.path.join(path_dmt_oc,'dim_curso_perfil.csv'))

In [ ]:
df_merge = pd.merge(df_fact_oc,df_dim_razon_social_oc,on='id_oc',how='left')
df_merge = pd.merge(df_merge,df_dim_curso_perfil,on='id_curso_perfil',how='left')

In [ ]:
df_query = df_merge[(df_merge['area_familia']=='información y comunicación')|(df_merge['area_familia'] == 'tecnologías de la información y comunicación')]

In [ ]:
df_query = df_query.sort_values(by='volumen_capacitados',ascending=False)

In [ ]:
df_query = df_query.drop_duplicates(subset=['nombre'])

In [ ]:
df_query

In [ ]:
df_query = df_query[['volumen_capacitados','nombre']]
df_query.to_csv('top_ocs.csv',index=False)

# Analisis OEC

In [ ]:
df_fact_oec = pd.read_csv(os.path.join(path_dmt_oec,'fact_oec.csv'))
df_dim_razon_social_oec = pd.read_csv(os.path.join(path_dmt_oec,'dim_razon_social_oec.csv'))
df_dim_perfil = pd.read_csv(os.path.join(path_dmt_oec,'dim_perfil.csv'))

In [ ]:
df_merge = pd.merge(df_fact_oec,df_dim_razon_social_oec,on='id_oec',how='left')
df_merge = pd.merge(df_merge,df_dim_perfil,on='id_perfil',how='left')

In [ ]:
df_query = df_merge[(df_merge['familia']=='tecnología: hardware y software (incluye tic)')]

In [ ]:
df_query = df_query.sort_values(by='volumen_capacitados',ascending=False)

In [ ]:
df_query = df_query.drop_duplicates(subset=['razon_social'],keep='first')

In [ ]:
df_query

In [ ]:
df_query = df_query[['volumen_capacitados','razon_social']]
df_query.to_csv('top_oecs.csv',index=False)

# Análisis CI

In [ ]:
df_fact_ci = pd.read_csv(os.path.join(path_dmt_ci,'fact_ci.csv'))
df_dim_capacitador_independiente = pd.read_csv(os.path.join(path_dmt_ci,'dim_capacitador_independiente.csv'))
df_dim_curso = pd.read_csv(os.path.join(path_dmt_ci,'dim_curso.csv'))

In [ ]:
df_merge = pd.merge(df_fact_ci,df_dim_capacitador_independiente,left_on='id_ci_y',right_on='id_ci',how='left')
df_merge = pd.merge(df_merge,df_dim_curso,on='id_curso',how='left')

In [ ]:
df_query = df_merge[(df_merge['area']=='tecnologías de la información y comunicación')]

In [ ]:
df_query = df_query.sort_values(by='volumen_capacitados',ascending=False)

In [ ]:
df_query = df_query.drop_duplicates(subset=['razon_social'],keep='first')

In [ ]:
df_query = df_query[['volumen_capacitados','razon_social']]
df_query.to_csv('top_cis.csv',index=False)

# Añadir KPI 5 OC

In [ ]:
df_fact_oc = pd.read_csv(os.path.join(path_dmt_oc,'fact_oc.csv'))
df_dim_razon_social_oc = pd.read_csv(os.path.join(path_dmt_oc,'dim_razon_social_oc.csv'))

In [ ]:
df_oc_num_conv= pd.read_csv(os.path.join(os.getcwd(),'oc_num_conv.csv'))
df_oc_num_conv.drop('volumen_capacitados',axis=1,inplace=True)

In [ ]:
df_merge = pd.merge(df_fact_oc,df_dim_razon_social_oc,on='id_oc',how='left')

In [ ]:
df_merge = pd.merge(df_merge,df_oc_num_conv,on='nombre',how='left')

In [ ]:
df_merge.columns

Index(['id_oc', 'id_curso_perfil', 'id_fecha', 'num_cap_cer', 'total_cursos',
       'volumen_capacitados', 'total_horas', 'total_ganancias', '%_asistencia',
       'nombre', 'estado', 'canton', 'provincia', ' facebook_seguidores',
       'instagram_seguidores', 'numero_convocados'],
      dtype='object')

In [ ]:
df_merge['%_asistencia'] = df_merge['num_cap_cer']/df_merge['numero_convocados']*100

In [ ]:
df_merge = df_merge[['id_oc', 'id_curso_perfil', 'id_fecha', 'num_cap_cer', 'total_cursos',
       'volumen_capacitados', 'total_horas', 'total_ganancias','%_asistencia']]

In [ ]:
df_merge

,id_oc,id_curso_perfil,id_fecha,num_cap_cer,total_cursos,volumen_capacitados,total_horas,total_ganancias,%_asistencia
0,NaN,NaN,3,17,2,63,NaN,NaN,NaN
1,NaN,NaN,3,46,2,63,NaN,NaN,NaN
2,369.0,NaN,2,2,4,52,NaN,NaN,NaN
3,1128.0,NaN,2,2,4,52,NaN,NaN,NaN
4,369.0,NaN,2,15,4,52,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9381,1207.0,NaN,2,8,7,206,NaN,NaN,NaN
9382,459.0,NaN,2,7,7,206,NaN,NaN,NaN
9383,1207.0,NaN,2,7,7,206,NaN,NaN,NaN
9384,459.0,NaN,2,8,7,206,NaN,NaN,NaN


In [ ]:
df_merge.to_csv(os.path.join(path_dmt_oc,'fact_oc.csv'),index=False)

# Añadir KPI5 OEC

In [ ]:
df_fact_oec = pd.read_csv(os.path.join(path_dmt_oec,'fact_oec.csv'))
df_dim_razon_social_oec = pd.read_csv(os.path.join(path_dmt_oec,'dim_razon_social_oec.csv'))

In [ ]:
df_oec_num_conv= pd.read_csv(os.path.join(os.getcwd(),'pcoec_num_conv.csv'),encoding = "ISO-8859-1")
df_oec_num_conv.drop('volumen_capacitados',axis=1,inplace=True)

In [ ]:
df_merge = pd.merge(df_fact_oec,df_dim_razon_social_oec,on='id_oec',how='left')

In [ ]:
df_merge = pd.merge(df_merge,df_oec_num_conv,on='razon_social',how='left')

In [ ]:
df_merge['%_asistencia'] = df_merge['num_cer']/df_merge['numero_convocados']*100

In [ ]:
df_merge.columns

Index(['id_oec', 'id_perfil', 'id_fecha', 'num_cer', 'total_horas',
       'total_cursos', 'volumen_capacitados', 'total_ganacias', 'razon_social',
       'estado', 'canton', 'provincia', 'nombre_comercial', 'pagina_facebook',
       'seguidores_facebook', 'pagina_instagram', 'seguidores_instagram',
       'numero_convocados', '%_asistencia'],
      dtype='object')

In [ ]:
df_merge = df_merge[['id_oec', 'id_perfil', 'id_fecha', 'num_cer', 'total_horas',
       'total_cursos', 'volumen_capacitados', 'total_ganacias','%_asistencia']]

In [ ]:
df_merge.to_csv(os.path.join(path_dmt_oec,'fact_oec.csv'),index=False)